Serialization - saving, loading and checkpointing


But even with all this knowledge, we’re not ready to build a real machine learning system.
That’s because we haven’t yet covered how to save and load models
In reality, we often train a model on one device and then want to run it to make predictions on many devices simultaneously
In order for our models to persist beyond the execution of a single Python script,
we need mechanisms to save and `load NDArrays, gluon Parameters, and models themselves.`


In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
ctx = mx.cpu()

# Saving and loading NDArrays

### save and load a list of NDArrays for future use
### We prefer to use `ndarray.save` and `ndarray.load`


In [4]:
X = nd.ones((100, 100))
Y = nd.zeros((100, 100))
import os
os.makedirs('checkpoints',exist_ok=True)
filename="checkpoints/test1.params"
nd.save(filename,[X,Y])

### It’s just as easy to load a saved NDArray.

In [5]:
A, B = nd.load(filename)
print(A)
print(B)


[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
<NDArray 100x100 @cpu(0)>

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 100x100 @cpu(0)>


In [9]:
print(os.path.abspath('Gluon_save_modle.ipynb')) 

/home/qy/gluon-tutorials-zh/chapter_supervised-learning/Gluon_save_modle.ipynb


We can also save a dictionary where the keys are strings and the values are NDArrays.

In [12]:
mydict = {"X": X, "Y": Y}
filename = "checkpoints/test2.params"
nd.save(filename, mydict)

In [13]:
C = nd.load(filename)
print(C)

{'X': 
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
<NDArray 100x100 @cpu(0)>, 'Y': 
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 100x100 @cpu(0)>}


## Saving and loading the parameters of` gluon` models


### gluon wraps the NDArrays corresponding to model parameters in Parameter objects.
### We’ll often want to store and load an entire model’s parameters without having to individually extract or load the NDarrays from the Parameters via ParameterDicts in each block.
### gluon blocks make our lives very easy by providing a `.save_params()` and `.load_params()` methods.

In [14]:
num_hidden = 256
num_outputs = 1
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

### Now, let’s initialize the parameters by attaching an initializer and actually passing in a datapoint to induce shape inference

In [17]:
net.collect_params().initialize(mx.init.Normal(sigma=1.), ctx=ctx)
net(nd.ones((1, 100), ctx=ctx))

/home/qy/software/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:685: UserWarning: Parameter 'sequential0_dense0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/qy/software/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:685: UserWarning: Parameter 'sequential0_dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/qy/software/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:685: UserWarning: Parameter 'sequential0_dense1_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/qy/software/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/gluon/parameter.py:685: UserWarning: Parameter 'sequential0_dense1_bias' is already in


[[-2362.1072]]
<NDArray 1x1 @cpu(0)>

### Let’s save the parameters, instantiate a new network, load them in and make sure that we get the same result.

In [18]:
filename = "checkpoints/testnet.params"
net.save_params(filename)
net2 = gluon.nn.Sequential()
with net2.name_scope():
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net2.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net2.add(gluon.nn.Dense(num_outputs))
net2.load_params(filename, ctx=ctx)
net2(nd.ones((1, 100), ctx=ctx))


[[-2362.1072]]
<NDArray 1x1 @cpu(0)>

### The practice of saving models is sometimes called checkpointing
## it’s especially important for a number of reasons. 
### 1. We can preserve and syndicate models that are trained once.
### 2. Some models perform best (as determined on validation data) at some epoch in the middle of training. If we checkpoint the model after each epoch, we can later select the best epoch.
### 3. We might want to ask questions about our trained model that we didn’t think of when we first wrote the scripts for our experiments. Having the parameters lying around allows us to examine our past work without having to train from scratch. 
### 4. Sometimes people might want to run our models who don’t know how to execute training themselves or can’t access a suitable dataset for training. Checkpointing gives us a way to share our work with others.